In [1]:
import requests
import json
import numpy as np
import pandas as pd
import csv
import time
import time
from datetime import datetime

In [5]:
EMAIL = ""
PASSWORD = ""
URL_LOGIN = "https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/"
headers = {"email": EMAIL,"password":PASSWORD}

In [6]:
response = requests.request("GET", URL_LOGIN, headers = headers)

In [7]:
response_json = json.loads(response.text)

In [8]:
accessToken = response_json["data"][0]["accessToken"]

In [9]:
headers_token = {"accessToken": accessToken}

## 4.6.1 TIPOS DE DÍA

In [3]:
movimientos_df = pd.read_csv('movimientos_general.csv', sep=";", header=0)

In [11]:
movimientos_df = movimientos_df[['date','id_line']]
f = lambda x: x["date"].split("T",1)[0]
movimientos_df["date"] = movimientos_df.apply(f,axis=1)

##### Fecha mínima encontrada

In [12]:
fecha_minima = movimientos_df['date'].min().replace("-","")
fecha_minima

'20170401'

##### Fecha máxima encontrada

In [13]:
fecha_maxima = movimientos_df['date'].max().replace("-","")
fecha_maxima

'20190626'

#### Obtener el calendario

In [14]:
URL_CALENDAR = "https://openapi.emtmadrid.es/v1/transport/busemtmad/calendar/" + fecha_minima + "/" + fecha_maxima + "/"

In [15]:
response_calendar = requests.request("GET", URL_CALENDAR, headers = headers_token)

In [16]:
calendario = []
parsed = (json.loads(response_calendar.text)["data"])
for data in parsed:
    calendario.append([datetime.strptime(data["date"], "%d/%m/%Y %H:%M:%S").strftime("%Y-%m-%d"),data["dayType"]])

In [17]:
calendario = pd.DataFrame(calendario,columns =['date','day_type'])

In [18]:
calendario.head(3)

,date,day_type
0,2017-04-01,SA
1,2017-04-02,FE
2,2017-04-03,LA


##### Número de días

In [19]:
calendario.shape[0]

817

## 4.6.2 FRECUENCIAS

In [20]:
movimientos_df.drop_duplicates(subset=['date', 'id_line'],inplace=True)

In [21]:
mov_linea_fecha = pd.merge(movimientos_df, calendario, on='date',how="left")

##### Movimientos únicos entre una línea y una fecha en específico de todo el período

In [22]:
mov_linea_fecha.shape[0]

6967

In [21]:
mov_linea_fecha.sample(5)

,date,id_line,day_type
3677,2018-06-17,361,FE
6657,2019-05-26,601,FE
4932,2018-11-11,46,FE
6445,2019-05-14,43,LA
3129,2018-04-08,69,FE


### 4.6.2.1 Análisis de una petición

In [331]:
line_o_label = "3"
dateref = "20170413"
URL_INFO_DETAIL = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/" + line_o_label + "/info/" + dateref + "/"

In [332]:
response_info_detail = requests.request("GET", URL_INFO_DETAIL, headers = headers_token)

In [333]:
parsed = (json.loads(response_info_detail.text)["data"][0])
print(json.dumps(parsed, indent=4))

{
    "line": "003",
    "nameA": "PUERTA TOLEDO",
    "nameB": "SAN AMARO",
    "dateRef": "13/04/2017",
    "label": "3",
    "timeTable": [
        {
            "idDayType": "FE",
            "Direction1": {
                "StartTime": "07:30",
                "StopTime": "23:30",
                "MinimunFrequency": "11",
                "MaximumFrequency": "34",
                "FrequencyText": "De 07:30 a 23:30 -> Cada 11 - 34min./"
            },
            "Direction2": {
                "StartTime": "07:00",
                "StopTime": "23:00",
                "MinimunFrequency": "11",
                "MaximumFrequency": "34",
                "FrequencyText": "De 07:00 a 23:00 -> Cada 11 - 34min./"
            }
        },
        {
            "idDayType": "LA",
            "Direction1": {
                "StartTime": "06:30",
                "StopTime": "23:30",
                "MinimunFrequency": "7",
                "MaximumFrequency": "14",
                "FrequencyTex

### 4.6.2.2 Conjunto de peticiones por mes

Convertimos la fecha a un formato en específico que la API requiere para pasar como parámetro

In [23]:
mov_linea_fecha['date'] = pd.to_datetime(mov_linea_fecha['date'], errors='coerce')

In [24]:
mov_linea_fecha['date_month'] = mov_linea_fecha['date'].dt.year.astype(str) + "-" + mov_linea_fecha['date'].dt.month.map("{:02}".format)

In [25]:
mov_linea_fecha.head(3)

,date,id_line,day_type,date_month
0,2017-04-12,3,LA,2017-04
1,2017-04-23,3,FE,2017-04
2,2017-04-08,19,SA,2017-04


In [26]:
movs_totales = mov_linea_fecha.groupby(['date_month','id_line']).count().reset_index()[['date_month','id_line']]
movs_totales['date_month_api'] = movs_totales['date_month'].astype(str).str.replace("-","")

##### Servicios durante el período

In [29]:
movs_totales.count()[0]

323

In [27]:
movs_totales.head(3)

,date_month,id_line,date_month_api
0,2017-04,3,201704
1,2017-04,19,201704
2,2017-04,26,201704


##### Obtener las frecuencias

In [66]:
def get_frecuencias(data,row):
    aux = []
    columns=['id_line','date_month','type_date','MinimunFrequency1','MaximumFrequency1','MinimunFrequency2','MaximumFrequency2']
    try:
        data = data["data"][0]["timeTable"]
        if type(data) == list: 
            for d in data:
                data = [[row["id_line"],row["date_month"],d["idDayType"],d["Direction1"]["MinimunFrequency"],d["Direction1"]["MaximumFrequency"],d["Direction2"]["MinimunFrequency"],d["Direction2"]["MaximumFrequency"]]]
                aux.append(pd.DataFrame(data,columns = columns))
            return aux
        else:
            return pd.DataFrame([[row["id_line"],row["date_month"],data["idDayType"],data["Direction1"]["MinimunFrequency"],data["Direction1"]["MaximumFrequency"],data["Direction2"]["MinimunFrequency"],data["Direction2"]["MaximumFrequency"]]],columns = columns)
    except:
         return [pd.DataFrame([[row["id_line"],row["date_month"],-1,-1,-1,-1,-1]],columns = columns)]
    return [pd.DataFrame([[row["id_line"],row["date_month"],-1,-1,-1,-1,-1]],columns = columns)]

In [69]:
import random

In [67]:
def get_frecuencias_total(df,dia):
    cont = pd.DataFrame()
    for index, row in df.iterrows():
        URL_INFO_DETAIL = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/" + str(row["id_line"]) + "/info/" + row["date_month_api"] + dia + "/"
        response_info_detail = requests.request("GET", URL_INFO_DETAIL, headers = headers_token)
        parsed = (json.loads(response_info_detail.text))
        res = get_frecuencias(parsed,row)
        time.sleep(round(random.uniform(0, 1),1))
        cont = cont.append(res)
    return cont

Le pasamos a la función el día sobre el cual realizará la búsqueda. Para este caso, será para el día 02 de cada "añomes"

In [70]:
cont = get_frecuencias_total(movs_totales,"02")

In [71]:
cont.sample(3)

,id_line,date_month,type_date,MinimunFrequency1,MaximumFrequency1,MinimunFrequency2,MaximumFrequency2
0,69,2018-04,SA,6,17,6,17
0,43,2017-08,SA,24,33,24,33
0,19,2019-05,SA,9,20,9,20


In [357]:
cont.to_csv('frecuencias_tipo_dia.csv',sep=";",header=['id_line','date_month','type_date','dir1_min_freq','dir1_max_freq','dir2_min_freq','dir2_max_freq'],index=False)

In [92]:
frecuencias_df = pd.read_csv('frecuencias_tipo_dia.csv', sep=";", header=0)

##### Revisamos si hay alguna línea donde no se haya obtenido registro en algún mes en específico

In [56]:
frecuencias_df[(frecuencias_df.dir1_min_freq == -1) | (frecuencias_df.dir1_max_freq == -1)].groupby('id_line').count()

,date_month,type_date,dir1_min_freq,dir1_max_freq,dir2_min_freq,dir2_max_freq
id_line,,,,,,
361,27,27,27,27,27,27
362,27,27,27,27,27,27


### 4.6.2.3 Líneas 361 y 362

In [64]:
lineas_no_buscadas = [['2020-03', '361','202003'], ['2020-03', '362','202003']] 
lineas_no_buscadas_df = pd.DataFrame(lineas_no_buscadas, columns = ['date_month','id_line','date_month_api']) 

In [70]:
cont2 = get_frecuencias_total(lineas_no_buscadas_df,"18")
cont2.columns = ['id_line','date_month','type_date','dir1_min_freq','dir1_max_freq','dir2_min_freq','dir2_max_freq']

In [71]:
cont2

,id_line,date_month,type_date,dir1_min_freq,dir1_max_freq,dir2_min_freq,dir2_max_freq
0,361,2020-03,FE,10,30,10,30
0,361,2020-03,LA,7,23,7,23
0,361,2020-03,SA,10,30,10,30
0,362,2020-03,FE,16,30,16,30
0,362,2020-03,LA,11,24,11,24
0,362,2020-03,SA,16,30,16,30


##### Eliminamos esas lineas

In [72]:
frecuencias_df = frecuencias_df[(frecuencias_df.dir1_min_freq != -1) | (frecuencias_df.dir1_max_freq != -1)]

##### Unimos ambos datasets

In [73]:
frecuencias_df = frecuencias_df.append(cont2)

In [74]:
frecuencias_df.to_csv('frecuencias_tipo_dia.csv',sep=";",index=False)

##### Número de frecuencias encontradas

In [86]:
frecuencias_df.count()[0]

774

### 4.6.2.4 Análisis de la dirección

In [99]:
frecuencias_df['diff_min'] = np.where(frecuencias_df['dir1_min_freq'] != frecuencias_df['dir2_min_freq'] , 1, 0)
frecuencias_df['diff_max'] = np.where(frecuencias_df['dir1_max_freq'] != frecuencias_df['dir2_max_freq'] , 1, 0)

##### tiempo mínimo 

In [100]:
diff_df = frecuencias_df[frecuencias_df['diff_min'] == 1]
diff_df

,id_line,date_month,type_date,dir1_min_freq,dir1_max_freq,dir2_min_freq,dir2_max_freq,diff_min,diff_max


##### tiempo máximo

In [101]:
diff_df = frecuencias_df[frecuencias_df['diff_max'] == 1]
diff_df

,id_line,date_month,type_date,dir1_min_freq,dir1_max_freq,dir2_min_freq,dir2_max_freq,diff_min,diff_max


#### CONCLUSIÓN: Son los mismos tiempos en ambos sentidos

##### Eliminamos una direción y los campos generados de las comparaciones

In [104]:
frecuencias_df.drop(['dir2_min_freq','dir2_max_freq','diff_min','diff_max'], axis=1, inplace=True)

In [105]:
frecuencias_df.head(5)

,id_line,date_month,type_date,dir1_min_freq,dir1_max_freq
0,3,2017-04,FE,11,34
1,3,2017-04,LA,7,14
2,3,2017-04,SA,10,23
3,19,2017-04,FE,14,30
4,19,2017-04,LA,3,15


##### Reescribimos el conjunto de datos

In [106]:
frecuencias_df.to_csv('frecuencias_tipo_dia.csv',sep=";",index=False)

### 4.6.2.5 Asignar frecuencias

##### Creamos una columna clave para relacionar las frecuencias con los registros de mov_linea_fecha

In [109]:
frecuencias_df['month_line_day_type'] = frecuencias_df['date_month'].astype(str).str.replace("-","") + frecuencias_df['type_date'] + frecuencias_df['id_line'].astype(str)

In [110]:
frecuencias_df = frecuencias_df[['dir1_min_freq','dir1_max_freq','month_line_day_type']]

In [158]:
frecuencias_df.head(3)

,dir1_min_freq,dir1_max_freq,month_line_day_type
0,11,34,201704FE3
1,7,14,201704LA3
2,10,23,201704SA3


In [111]:
mov_linea_fecha['month_line_day_type'] = mov_linea_fecha['date_month'].astype(str).str.replace("-","") + mov_linea_fecha['day_type'] + mov_linea_fecha['id_line'].astype(str)

In [39]:
mov_linea_fecha.head(3)

,date,id_line,day_type,date_month,month_line_day_type
0,2017-04-12,3,LA,2017-04,201704LA3
1,2017-04-23,3,FE,2017-04,201704FE3
2,2017-04-08,19,SA,2017-04,201704SA19


##### Relacionamos ambos datasets

In [112]:
frecuencias_df_new =  mov_linea_fecha.merge(frecuencias_df, on='month_line_day_type',how='left')

##### Número de registros

In [114]:
frecuencias_df_new.count()[0]

6967

#### Revisamos si hay alguna línea donde no se haya obtenido registro

In [145]:
frecuencias_df_new[frecuencias_df_new.dir1_min_freq.isna()].groupby('id_line').count()

,date,day_type,date_month,month_line_day_type,dir1_min_freq,dir1_max_freq
id_line,,,,,,
156,34,34,34,34,0,0
361,773,773,773,773,0,0
362,655,655,655,655,0,0
601,142,142,142,142,0,0


Por el momento no consideramos la 361 y 362. Analizaremos la 156 y 601

### 4.6.2.6 Días festivos como sábado

In [147]:
frecuencias_df_new[(frecuencias_df_new.dir1_min_freq.isna()) & (frecuencias_df_new['id_line'].isin([156,601]))].groupby('day_type').count()

,date,id_line,date_month,month_line_day_type,dir1_min_freq,dir1_max_freq
day_type,,,,,,
FE,176,176,176,176,0,0


Como solo se presentan frecuencias nulas para los días 'FE' (Festivos), para proseguir con el estudio, asumiremos que un viernes se comporta como un sábado. 

In [115]:
festivo_a_sabado = frecuencias_df_new[(frecuencias_df_new.dir1_min_freq.isna()) & (frecuencias_df_new['id_line'].isin([156,601]))]

In [67]:
festivo_a_sabado.head(1)

,date,id_line,day_type,date_month,month_line_day_type,dir1_min_freq,dir1_max_freq
139,2017-04-02,156,FE,2017-04,201704FE156,NaN,NaN


##### Sustitución de frecuencias del tipo de día 'FE' por frecuencias del tipo de día 'SA'

In [69]:
def get_frecuencias_NaN(data,day_type):
    try:
        for d in data["data"][0]["timeTable"]:
            if d["idDayType"] == "SA":
                return [d["Direction1"]["MinimunFrequency"],d["Direction1"]["MaximumFrequency"]]
    except:
         return [-1,-1]
    return ["",""]

In [85]:
f_s = []
for index, row in festivo_a_sabado.iterrows():
    URL_INFO_DETAIL = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/" + str(row["id_line"]) + "/info/" + str(row["date_month"].replace("-","")) + "02/"
    response_info_detail = requests.request("GET", URL_INFO_DETAIL, headers = headers_token)
    parsed = (json.loads(response_info_detail.text))
    res = get_frecuencias_NaN(parsed,row["day_type"])
    result = [row["date"],row["id_line"],row["day_type"],row["date_month"],row["month_line_day_type"],res[0],res[1]]
    f_s.append(result)

In [87]:
festivo_a_sabado_new = pd.DataFrame(cont_NaN,columns=['date','id_line','day_type','date_month','month_line_day_type','dir1_min_freq','dir1_max_freq'])

In [88]:
festivo_a_sabado_new.head(1)

,date,id_line,day_type,date_month,month_line_day_type,dir1_min_freq,dir1_max_freq
0,2017-04-02,156,FE,2017-04,201704FE156,13,28


##### Revisamos si hay nulos

In [89]:
festivo_a_sabado_new[festivo_a_sabado_new['dir1_min_freq'].isna()].groupby('id_line').count()

,date,day_type,date_month,month_line_day_type,dir1_min_freq,dir1_max_freq
id_line,,,,,,


### 4.6.2.7 Agregación de frecuencias 

##### Agregamos los registros de las líneas 361 y 362

In [243]:
cont2_copy = cont2[['id_line','type_date','dir1_min_freq','dir1_max_freq']]
cont2_copy.columns = ['id_line','day_type','dir1_min_freq','dir1_max_freq']
cont2_copy['line_day_type'] = cont2_copy['id_line'].astype(str) +  cont2_copy['day_type']
cont2_copy.drop(['id_line','day_type'], axis=1, inplace=True)

C:\Users\Arturo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Arturo\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [244]:
cont2_copy

,dir1_min_freq,dir1_max_freq,line_day_type
0,10,30,361FE
0,7,23,361LA
0,10,30,361SA
0,16,30,362FE
0,11,24,362LA
0,16,30,362SA


In [121]:
lineas_361_362 = frecuencias_df_new[frecuencias_df_new['id_line'].isin([361,362])][['date','id_line','day_type','date_month','month_line_day_type']]

In [268]:
lineas_361_362['line_day_type'] = lineas_361_362['id_line'].astype(str) +  lineas_361_362['day_type']

In [269]:
lineas_361_362.head(3)

,date,id_line,day_type,date_month,month_line_day_type,line_day_type
154,2017-04-01,361,SA,2017-04,201704SA361,361SA
155,2017-04-03,361,LA,2017-04,201704LA361,361LA
156,2017-04-05,361,LA,2017-04,201704LA361,361LA


##### Número de registros a modificar

In [124]:
lineas_361_362.count()[0]

1428

In [270]:
lineas_361_362_new = pd.merge(lineas_361_362, cont2_copy, on='line_day_type',how="left")

In [271]:
lineas_361_362_new.head(3)

,date,id_line,day_type,date_month,month_line_day_type,line_day_type,dir1_min_freq,dir1_max_freq
0,2017-04-01,361,SA,2017-04,201704SA361,361SA,10,30
1,2017-04-03,361,LA,2017-04,201704LA361,361LA,7,23
2,2017-04-05,361,LA,2017-04,201704LA361,361LA,7,23


In [273]:
lineas_361_362_new.drop('line_day_type', axis=1, inplace=True)

##### Eliminamos los registros completos

In [274]:
frecuencias_df_new = frecuencias_df_new[(frecuencias_df_new['dir1_min_freq'].notna()) | (frecuencias_df_new['dir1_max_freq'].notna())]

##### Agregamos los registros de festivo a sábado

In [275]:
frecuencias_df_new = frecuencias_df_new.append(festivo_a_sabado_new)

##### Agregamos los registros de las lineas especiales 361 y 362

In [276]:
frecuencias_df_new = frecuencias_df_new.append(lineas_361_362_new)

### 4.6.2.8 Asignar frecuencias a los movimientos en bicicleta

In [307]:
movimientos_df = pd.read_csv('movimientos_general_agregado.csv', sep=";", header=0)

f = lambda x: x["date"].split("T",1)[0].replace("-","") + str(x["id_line"])
movimientos_df["dateline_join"] = movimientos_df.apply(f,axis=1)

f2 = lambda x: x["date"].strftime("%Y%m%d") + str(x["id_line"])
frecuencias_df_new["dateline_join"] = frecuencias_df_new.apply(f2,axis=1)
frecuencias_df_new.sort_values(['id_line','date'],inplace=True)

In [70]:
movimientos_df.head(2)

,user_day_code,user_type,travel_time,idunplug_station,ageRange,idplug_station,date,id_line,id_header_A,id_header_B,direction,dateline_join
0,5876778a2a6a59f97028d504c89d516e36bf94e6480b4f...,1,1672,141,4,49,2017-04-12T17:00:00.000+0200,3,1885,1855,AtoB,201704123
1,1c7b8e5eb895cb7268cbf215cfe09598d557711ec6b839...,1,1724,49,4,141,2017-04-23T16:00:00.000+0200,3,1885,1855,BtoA,201704233


In [308]:
frecuencias_df_new.head(3)

,date,id_line,day_type,date_month,month_line_day_type,dir1_min_freq,dir1_max_freq,dateline_join
0,2017-04-12,3,LA,2017-04,201704LA3,7,14,201704123
1,2017-04-23,3,FE,2017-04,201704FE3,11,34,201704233
238,2017-05-05,3,LA,2017-05,201705LA3,7,14,201705053


In [314]:
movimientos_df_new =  movimientos_df.merge(frecuencias_df_new, on='dateline_join',how='inner')
movimientos_df_new.rename(columns={'date_x': 'date', 'id_line_x': 'id_line'}, inplace=True)
movimientos_df_new.drop(['id_line_y','date_y','dateline_join','date_month','month_line_day_type'], axis=1, inplace=True)

In [311]:
movimientos_df_new.shape

(26737, 16)

In [315]:
movimientos_df_new.sample(3)

,user_day_code,user_type,travel_time,idunplug_station,ageRange,idplug_station,date,id_line,id_header_A,id_header_B,direction,day_type,dir1_min_freq,dir1_max_freq
20896,c46b1413362b6f9277b2aba16406d1ac62d16a23518f45...,1,1359,148,4,135,2019-02-02T13:00:00.000+0100,19,5167,1171,BtoA,SA,9,20
5421,d1812ed2c52080b718df3270a2ea47d7f39ec9842039f2...,1,1021,49,4,131,2017-09-14T13:00:00.000+0200,362,80,289,BtoA,LA,11,24
589,43c250b5b829bc19f8ba905a6f9171aad52670fec38083...,1,5734,53,3,1,2017-04-15T00:00:00.000+0200,601,5837,4057,AtoB,SA,12,20


##### Revisamos si hay nulos

In [129]:
movimientos_df_new[(movimientos_df_new['dir1_min_freq'].isna()) | (movimientos_df_new['dir1_max_freq'].isna())].groupby('id_line').count()

,user_day_code,user_type,travel_time,idunplug_station,ageRange,idplug_station,date,id_header_A,id_header_B,direction,day_type,dir1_min_freq,dir1_max_freq
id_line,,,,,,,,,,,,,


In [316]:
movimientos_df_new.to_csv('movimientos_general_con_frecuencias.csv',sep=";",index=False)